In [ ]:
import os, sys, pickle, copy
import numpy as np
import scipy as sp
from scipy.constants import Avogadro, R
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb, rgb_to_hsv, to_rgb
from matplotlib.patches import FancyArrowPatch, ArrowStyle
from datetime import datetime

# import os
# os.add_dll_directory(os.getcwd())

from setup_simulation import *

# Load data from our previous paper on this topic:
# J Physiol 601.24 (2023) pp 5705–5732
# https://doi.org/10.1113/JP284872
#power demands of motor nerve terminals
terminal_data = load_terminal_data_xlsx('terminal_data.xlsx')
terminals     = sorted(terminal_data.keys())[::-1]
terminals     = terminals[1::2]+terminals[::2]

## Prepare simulations

In [ ]:
# ---------- Output options ----------

data_dir = 'figure-data'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

fig_dir = 'figures'
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)

cm = 1/2.54 # Inch-to-cm conversion: 1*cm is 1cm in inches.

def data_file(figname): #, **kwargs):
    fn = ( f'{data_dir}/{figname}_{terminal}_conc={concentration_set}'
           f'_prod={production_model}_rateCapPpc={rate_cap_per_pct_mito}' )
    return fn + '.npz'

def savefig(filename):
    fn = f'{fig_dir}/{filename}'
    plt.savefig(fn+'.png', dpi=250)
    plt.savefig(fn+'.pdf')
    plt.savefig(fn+'.eps')

# ---------- Plotting options ----------

plt.rcParams.update({
    'savefig.facecolor'    : 'w',
    'lines.linewidth'      : 0.5,
    'lines.linewidth'      : 0.5,
    'font.size'            : 7,
    'legend.frameon'       : False,
    'legend.loc'           : 'upper right',
    'legend.fontsize'      : 7,
    'legend.handlelength'  : 1,
    'legend.labelspacing'  : 0.25,
    'legend.borderpad'     : 0,
    'legend.borderaxespad' : 0,
})

colors    = { 'M6Ib':'#009b00', 'M6Is':'#ff0000', 'M13Ib':'#0000ff',
              'M13Is':'#ffa500', 'M12Ib':'#ff00ff', 'M12Is':'#c8c800' }
zorders   = { term:i for i,term in enumerate(['M12Ib','M12Is','M13Is',
                                              'M6Is','M6Ib','M13Ib'], 1) }
term_opt  = lambda term: dict(color=colors[term], label=term[1:])

panel_opt = dict(x=-0.1, y=1.1, ha='right', va='bottom',
                 fontdict={'fontsize':10, 'weight':'bold'} )

# ---------- Simulation wrapper ----------

def run_simulation(settings, figname, recompute_all=False,
                   verbose=True):
    t0   = datetime.now()
    globals().update(settings)
    fn = data_file(figname)
    if verbose:
        print(fn)
    if not recompute_all and os.path.exists(fn):
        if verbose:
            print(indent+'[already exists]')
        return False
    globals().update(setup(settings))
#     return settings
    data = integrate(verbose=verbose).T
    if verbose:
        print(indent+'[done in ' +f'{datetime.now()-t0}'[:-4] +'s]')
    np.savez(fn, settings=settings, data=data, times=times,
             consumption=consumption)
    return True


In [ ]:
caps = [0.032, 0.079, 0.154]


## Figure 7 modeling, M13b

First block simulates the M13b terminal. Save data as `fig7_13b_80Hz10sec[...].npz`.


In [ ]:
## Define models for particular figure ##
models_fig7_13b = [{'concentration_set': 'squid axon',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[1],
  'model_name': 'Squid axon\nmax prod 0.079 mM/s/%'},
  {'concentration_set': 'squid axon',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[2],
  'model_name': 'Squid axon\nmax prod 0.154 mM/s/%'},
{'concentration_set': 'squid axon no arginine',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[1],
  'model_name': 'Squid axon, no arginine\nmax prod 0.079 mM/s/%'},
{'concentration_set': 'squid axon no arginine',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[2],
  'model_name': 'Squid axon, no arginine\nmax prod 0.154 mM/s/%'}]
## Pick Terminal types to apply models to
terminalsfig7 = ['M6Ib','M6Is','M13Ib']

## Name figure
fn_fig7_13b = 'fig7_13b_80Hz10sec'

## Give time of model in minutes
num_mins      = 2/3  #40 seconds

## Generate data for above parameters
for terminal in [terminalsfig7[-1]]:
    for model in models_fig7_13b:
        globals().update(model)
        settings = create_settings(concentration_set, terminal, production_model,
                                   rate_cap_per_pct_mito=rate_cap_per_pct_mito,
                                   t_active=10, t_range=[-1,num_mins*60])
        globals().update(settings)
        settings['firing_rate']     = 80
        settings['firing_period']   = 1.0/settings['firing_rate']
        settings['duty_cycle']      = 1
        settings['full_cycle']      = 1
        settings['env_cycle']       =  None
        settings['duty_env_cycle']  =  None
        globals().update(settings)
        run_simulation(settings, fn_fig7_13b, verbose=True, recompute_all=False)

Now, use the npz files to make the figure. Save as `fig7_13b_80Hz10sec[...].eps` and `fig7_13b_80Hz10sec[...].png`.

In [ ]:
for figname,models_ in [ (fn_fig7_13b,models_fig7_13b)]:
## Set Figure parameters, spacing, etc.
    n_row    = 6
    n_col    = int(len(models_)/2)
    gridspec = dict(hspace=0.55, wspace=0.55, left=0.08,
                    right=0.98, bottom=0.07, top=0.95)
    fig,axs  = plt.subplots(n_row, n_col, figsize=(13.1*cm,3.5*n_row*cm),
                            gridspec_kw=gridspec)

    for i,terminal in enumerate([terminalsfig7[-1]]):
        for j,model in enumerate(models_):
            ## Load simulated data and model parameters
            globals().update(model)
            globals().update(np.load(data_file(figname), allow_pickle=True))
            globals().update(settings.item())
            df = dict(np.load(data_file(figname), allow_pickle=True))
            x1 = -5
            I = (data[0]>=x1+2) & (data[0]<=num_mins*60)
            T,Rp,ADP,ATP,ARG = data[[0,2,3,4,5]][:,I]  # extracts from loaded data for metabolite concentrations and Rate of production RP
            pi = Pt - ADP - 2*ATP - (Pht - ARG )  #obtains [Pi] by constraint
            ΔG0 = -30.6e3 # ΔG under standard conditions ([ATP]=[ADP]=[P]=1M), in J/mol
            Tmp = 300     # temperature in K
            ΔG  = ΔG0 + R*Tmp*np.log((ADP*10**-3)*(pi*10**-3)/(ATP*10**-3)) #Calculate ATP Hydrolysis Energy
            ΔG *= 1e-3

            style = ArrowStyle('wedge', tail_width=1.5, shrink_factor=0.5)   # Instantiate production cap marker
            opt   = dict(mutation_scale=1, arrowstyle=style, color=colors[terminal])
            dx    = 4
            ## Assign columns as either 'with' (k=0) or 'without phosphagen' 
            if Kph == None:
                k = 1
            else:
                k = 0            
            ## Plot Curves
            if rate_cap_per_pct_mito==caps[2]:
                axs[1,k].plot(T, ATP, **term_opt(terminal))
                axs[3,k].add_patch(FancyArrowPatch((0,rate_cap), (min(T)/2,rate_cap), **opt)) #Place Cap Marker
                axs[2,k].plot(T,ATP/ADP,**term_opt(terminal))
                axs[3,k].plot(T, Rp, **term_opt(terminal))
                acc = np.gradient(Rp, T)
                axs[4,k].plot(T, acc, **term_opt(terminal))
                axs[1,k].set_ylim(*ATP[0]*np.array([-0.,1.2]))
                axs[2,k].set_ylim(*(ATP/ADP)[0]*np.array([-0.,1.2]))
                axs[4,k].plot(T, acc, **term_opt(terminal))
                axs[5,k].plot(T, -ΔG, **term_opt(terminal))        
            elif rate_cap_per_pct_mito==caps[1]:
                axs[1,k].plot(T, ATP, **{'color': 'black', 'label': str(terminal) + ' 0.079 cap'})        
        I = times<=num_mins*60
        axs[0,0].set_ylim((0,1.5))
        axs[0,0].plot(times[I], consumption[I], **term_opt(terminal))
    ## Stylyze plots  
    for j in range(1,n_col):
        axs[0,j].axis('off')
    axs_ = axs[0,:1].tolist() + [ax for col in axs.T for ax in col[1:]]
    for i,ax in enumerate(axs_):
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)
        ax.set_xlabel('Time (s)')
        ax.locator_params(nbins=4)
        ax.text(s=chr(65+i), transform=ax.transAxes, **panel_opt)
        ax.set_xlim((-2,num_mins*60),None)
    axs[0,0].set_ylabel('ATP cons. rate (mM/s)')
    axs[0,0].legend(frameon=False)
    for j in range(axs.shape[1]):
        axs[1,j].set_ylabel('[ATP] (mM)')
        axs[2,j].set_ylabel('[ATP]/[ADP] ')
        axs[3,j].set_ylabel('ATP prod. rate \n(mM/s)')
        axs[4,j].set_ylabel('ATP prod. accel. \n(mM/s/s)')
        axs[5,j].set_ylabel('ATP hydrolysis\nenergy (kJ/mol)')        
    for i in [3,4,5]:
        ylim = list(zip(*[ax.get_ylim() for ax in axs[i]]))
        ylim = min((0,)+ylim[0]), max(ylim[1])
        for ax in axs[i]:
            ax.set_ylim(ylim)
    axs[0,0].set_yticks([0,0.5,1.0,1.5])
    axs[1,1].legend()
    axs[5,0].set_yticks([15,30,45,60])
    axs[5,1].set_yticks([15,30,45,60])
    axs[5,0].set_ylim(0,60)
    axs[5,1].set_ylim(0,60)    

    fig.suptitle('80 Hz Simulation 10sec' + figname)
    savefig(figname)
    plt.show()

    break

## Figure 7 modeling, M6Ib + M6Is 

First block simulates the M6Ib and M6Is terminals. Save data as `fig7_6bs_60Hzhalfsec[...].npz`.

In [ ]:
models_fig7_6bs = models_fig7_13b

fn_fig7_6bs = 'fig7_6bs_60Hzhalfsec'
num_mins      = 1/60 

for terminal in terminalsfig7[:-1]:
    for model in models_fig7_6bs:
        globals().update(model)
        settings = create_settings(concentration_set, terminal, production_model,
                                   rate_cap_per_pct_mito=rate_cap_per_pct_mito,
                                   t_active=1, t_range=[-0.1,1])
        settings['firing_rate']     = 60
        settings['firing_period']   = 1.0/settings['firing_rate']
        settings['duty_cycle']      = 30/60
        settings['full_cycle']      = 1
        settings['env_cycle']       =  None  #3
        settings['duty_env_cycle']  =  None  #2
        globals().update(settings)
        run_simulation(settings, fn_fig7_6bs, verbose=True, recompute_all=False)

Now, use the npz files to make the figure. Save as `fig7_6bs_60Hzhalfsec[...].eps` and `fig7_6bs_60Hzhalfsec[...].png`.

In [ ]:
for figname,models_ in [ (fn_fig7_6bs,models_fig7_6bs)]:

    n_row    = 6
    n_col    = int(len(models_)/2)
    gridspec = dict(hspace=0.55, wspace=0.55, left=0.08,
                    right=0.98, bottom=0.07, top=0.95)
    fig,axs  = plt.subplots(n_row, n_col, figsize=(13.1*cm,3.5*n_row*cm),
                            gridspec_kw=gridspec)

    for i,terminal in enumerate(terminalsfig7[:-1]):
        for j,model in enumerate(models_):
            globals().update(model)
            globals().update(np.load(data_file(figname), allow_pickle=True))
            globals().update(settings.item())
            df = dict(np.load(data_file(figname), allow_pickle=True))
            x1 = -5
            I = (data[0]>=x1+2) & (data[0]<=num_mins*60)
            T,Rp,ADP,ATP,ARG = data[[0,2,3,4,5]][:,I]
            pi = Pt - ADP - 2*ATP - (Pht - ARG )
            ΔG0 = -30.6e3 
            Tmp = 300 
            ΔG  = ΔG0 + R*Tmp*np.log((ADP*10**-3)*(pi*10**-3)/(ATP*10**-3))
            ΔG *= 1e-3 
            style = ArrowStyle('wedge', tail_width=1.5, shrink_factor=0.5)
            opt   = dict(mutation_scale=1, arrowstyle=style, color=colors[terminal])
            dx    = 4
            if Kph == None:
                k = 1
            else:
                k = 0            
            if rate_cap_per_pct_mito==caps[2]:
                axs[1,k].plot(T, ATP, **term_opt(terminal))
                axs[3,k].add_patch(FancyArrowPatch((0,rate_cap), (min(T)/2,rate_cap), **opt))
                axs[2,k].plot(T,ATP/ADP,**term_opt(terminal))
                axs[3,k].plot(T, Rp, **term_opt(terminal))
                acc = np.gradient(Rp, T)
                axs[4,k].plot(T, acc, **term_opt(terminal))
                axs[1,k].set_ylim(*ATP[0]*np.array([-0.,1.2]))
                axs[2,k].set_ylim(*(ATP/ADP)[0]*np.array([-0.,1.2]))
                axs[4,k].plot(T, acc, **term_opt(terminal))
                axs[5,k].plot(T, -ΔG, **term_opt(terminal))        
            elif rate_cap_per_pct_mito==caps[1] and terminal == 'M6Is':
                axs[1,k].plot(T, ATP, **{'color': 'black', 'label': str(terminal) + ' 0.079 cap'})
            elif rate_cap_per_pct_mito==caps[1] and terminal == 'M6Ib':
                axs[1,k].plot(T, ATP, **{'color': 'grey', 'label': str(terminal) + ' 0.079 cap'})                
        I = times<=num_mins*60

        axs[0,0].set_ylim((0,1.5))
        axs[0,0].plot(times[I], consumption[I], **term_opt(terminal))
    for j in range(1,n_col):
        axs[0,j].axis('off')
    axs_ = axs[0,:1].tolist() + [ax for col in axs.T for ax in col[1:]]
    for i,ax in enumerate(axs_):
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)
        ax.set_xlabel('Time (s)')
        ax.locator_params(nbins=4)
        ax.text(s=chr(65+i), transform=ax.transAxes, **panel_opt)
        ax.set_xlim((min(T)/2,max(T)),None)
    axs[0,0].set_ylabel('ATP cons. rate (mM/s)')
    axs[0,0].legend(frameon=False)
    for j in range(axs.shape[1]):
        axs[1,j].set_ylabel('[ATP] (mM)')
        axs[2,j].set_ylabel('[ATP]/[ADP] ')
        axs[3,j].set_ylabel('ATP prod. rate \n(mM/s)')
        axs[4,j].set_ylabel('ATP prod. accel. \n(mM/s/s)')
        axs[5,j].set_ylabel('ATP hydrolysis\nenergy (kJ/mol)')        
    for i in [3,4,5]:
        ylim = list(zip(*[ax.get_ylim() for ax in axs[i]]))
        ylim = min((0,)+ylim[0]), max(ylim[1])
        for ax in axs[i]:
            ax.set_ylim(ylim)
    axs[0,0].set_yticks([0,0.5,1.0,1.5,2,2.5])
    axs[5,0].set_yticks([15,30,45,60])    
    axs[5,1].set_yticks([15,30,45,60])    
    fig.suptitle(figname )
    savefig(figname)
    plt.show()

## Figure 6 modeling, J, L-P

First block simulates the M13b terminal. Save data as `fig6_50Hz'[...].npz`.


In [ ]:
caps

In [ ]:
models_fig6_2 = [
{'concentration_set': 'squid axon',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[1],
  'model_name': 'Squid axon\nmax prod 0.079 mM/s/%'},
    {'concentration_set': 'squid axon no arginine',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[1],
  'model_name': 'Squid axon, no arginine\nmax prod 0.079 mM/s/%'},
{'concentration_set': 'squid axon',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[2],
  'model_name': 'Squid axon\nmax prod 0.154 mM/s/%'},
{'concentration_set': 'squid axon no arginine',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[2],
  'model_name': 'Squid axon, no arginine\nmax prod 0.154 mM/s/%'}]

fn_fig6_2 = 'fig6_50Hz'


terminalsfig6_2 = ['M13Ib']
num_mins      =  4

for terminal in terminalsfig6_2:
    for model in models_fig6_2: 
        globals().update(model)
        settings = create_settings(concentration_set, terminal, production_model,
                                   rate_cap_per_pct_mito=rate_cap_per_pct_mito,
                                   t_active=60*num_mins, t_range=[-5,60*num_mins])
        globals().update(settings)
        settings['firing_rate']     =  50
        settings['firing_period']   =  1.0/settings['firing_rate']
        settings['duty_cycle']      =  1
        settings['env_cycle']       =  4
        settings['duty_env_cycle']  =  2
        globals().update(settings)
        run_simulation(settings, fn_fig6_2, verbose=True, recompute_all=False)


Now, use the npz files to make the figure. Save as `fig6_50Hz[...].eps` and `fig6_50Hz[...].png`.

In [ ]:
for figname,models_ in [ (fn_fig6_2,models_fig6_2)]:
    n_row    = 6
    n_col    = int(len(models_)/2)
    gridspec = dict(hspace=0.55, wspace=0.55, left=0.08,
                    right=0.98, bottom=0.07, top=0.95)
    fig,axs  = plt.subplots(n_row, n_col, figsize=(13.1*cm,3.5*n_row*cm),
                            gridspec_kw=gridspec)

    for i,terminal in enumerate(terminalsfig6_2):
        for j,model in zip([0,1]*2,models_):
            globals().update(model)
            globals().update(np.load(data_file(figname), allow_pickle=True))
            globals().update(settings.item())
            df = dict(np.load(data_file(figname), allow_pickle=True))
            x1 = -5
            I = (data[0]>=x1+2) & (data[0]<=60*num_mins)
            T,Rp,ADP,ATP,ARG = data[[0,2,3,4,5]][:,I]
            pi = Pt - ADP - 2*ATP - (Pht - ARG )  
            ΔG0 = -30.6e3 
            Tmp = 300 
            ΔG  = ΔG0 + R*Tmp*np.log((ADP*10**-3)*(pi*10**-3)/(ATP*10**-3))
            ΔG *= 1e-3
            if Kph == None:
                k = 1
            else:
                k = 0   
            if rate_cap_per_pct_mito == caps[2]:
                # Show production caps as arrow heads at appropriate height along the y axis.
                # If they're too large, have the arrow head point up and write the value.
                style = ArrowStyle('wedge', tail_width=1.5, shrink_factor=0.5)
                opt   = dict(mutation_scale=1, arrowstyle=style, color=colors[terminal])
                dx    = 10
                ax    = axs[3,j]
                ax.add_patch(FancyArrowPatch((x1+dx,rate_cap), (x1,rate_cap), **opt))
                ax.add_patch(FancyArrowPatch((dx,rate_cap), (0,rate_cap), **opt))
                axs[1,k].plot(T, ATP, **term_opt(terminal))
                axs[2,k].plot(T,ATP/ADP,**term_opt(terminal))
                axs[3,k].plot(T, Rp, **term_opt(terminal))
                acc = np.gradient(Rp, T)
                axs[4,k].plot(T, acc, **term_opt(terminal))
                axs[5,k].plot(T, -ΔG, **term_opt(terminal))  
                axs[1,k].set_ylim(*ATP[0]*np.array([-0.,1.2]))
                axs[2,k].set_ylim(*(ATP/ADP)[0]*np.array([-0.,1.2]))
            else:  ## Plots [ATP] for 0.079 mM/s/1% cap
                axs[1,j].plot(T, ATP, color='black')  
        I =  times<=num_mins*60
        I_sm = (times>=-0.2) & (times<=7.99)        
        axs[0,0].set_ylim((0,1.5))
        axs[0,0].plot(times[I], consumption[I], **term_opt(terminal))
        axs[0,1].plot(times[I_sm], consumption[I_sm], **term_opt(terminal))

    # for j in range(1,n_col):
    #     axs[0,j].axis('off')
#    axs_ = axs[0,:1].tolist() + [ax for col in axs.T for ax in col[1:]]
    axs_ = [ax for col in axs.T for ax in col[:]]
    for i,ax in enumerate(axs_):
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)
        ax.set_xlabel('Time (s)')
        ax.locator_params(nbins=4)
        ax.text(s=chr(65+i), transform=ax.transAxes, **panel_opt)
        ax.set_xlim( (-0.08,num_mins*60),None)
    axs[0,1].set_xlim((-0.2,max(times[I_sm] )) ,None)    
    axs[0,0].set_ylabel('ATP cons. rate (mM/s)')
    axs[0,1].set_ylabel('ATP cons. rate (mM/s)')
    axs[0,0].legend(frameon=False)
    for j in range(axs.shape[1]):
        axs[1,j].set_ylabel('[ATP] (mM)')
        axs[2,j].set_ylabel('[ATP]/[ADP] ')
        axs[3,j].set_ylabel('ATP prod. rate\n(mM/s)')
        axs[4,j].set_ylabel('ATP prod. accel.\n(mM/s/s)')
        axs[5,j].set_ylabel('ATP hydrolysis\nenergy (kJ/mol)')        
    for i in [3,4,5]:
        ylim = list(zip(*[ax.get_ylim() for ax in axs[i]]))
        ylim = min((0,)+ylim[0]), max(ylim[1])
        for ax in axs[i]:
            ax.set_ylim(ylim)
    axs[0,0].set_yticks([0,0.5,1.0,1.5])
    axs[0,1].set_xticks([0,2,4,6,8])
    axs[5,0].set_yticks([15,30,45,60])    
    axs[5,1].set_yticks([15,30,45,60])    
    fig.suptitle(figname +'50Hz 2on2off, w/ phos (left column) and w/o (right) phos')
    savefig(figname+' 240sec')
    plt.show()

    break

## ** WARNING ** Simulations for these two blocks may take as much as 30 hours!!   For endogenous firing protocols, with and without phos system:
 

## Figure 6 modeling, C-I

First block simulates the M13b terminal. Save data as `fig6_endogenous_rates[...].npz`.


In [ ]:
models_fig6_1 = [{'concentration_set': 'squid axon',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[2],
  'model_name': 'Squid axon, no arginine\nmax prod 0.154 mM/s/%'},
{'concentration_set': 'squid axon no arginine',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[2],
  'model_name': 'Squid axon\nmax prod 0.154 mM/s/%'},
{'concentration_set': 'squid axon',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[1],
  'model_name': 'Squid axon, no arginine\nmax prod 0.079 mM/s/%'},
{'concentration_set': 'squid axon no arginine',
  'production_model': 'wilson',
  'rate_cap_per_pct_mito': caps[1],
  'model_name': 'Squid axon\nmax prod 0.079 mM/s/%'}]



fn_fig6_1 = 'fig6_endogenous_rates'

terminalsfig6_1 = ['M6Ib','M13Ib']
num_mins      = 20 

for terminal in terminalsfig6_1:
    for model in models_fig6_1: #+models2:
        globals().update(model)
        settings = create_settings(concentration_set, terminal, production_model,
                                   rate_cap_per_pct_mito=rate_cap_per_pct_mito,
                                   t_active=np.inf, t_range=[-1,num_mins*60])
        globals().update(settings)
        settings['firing_period']   = 1.0/settings['firing_rate']
        settings['duty_cycle']      = 2
        settings['full_cycle']      = 3
        settings['env_cycle']       =  3  
        settings['duty_env_cycle']  =  2  
        globals().update(settings)
        run_simulation(settings, fn_fig6_1, verbose=True, recompute_all=False)


Now, use the npz files to make the figure. Save as `fig6_endogenous_rates[...].eps` and `fig6_endogenous_rates[...].png`.

In [ ]:
for figname,models_ in [ (fn_fig6_1,models_fig6_1)]:

    n_row    = 6
    n_col    = int(len(models_)/2)
    gridspec = dict(hspace=0.55, wspace=0.55, left=0.08,
                    right=0.98, bottom=0.07, top=0.95)
    fig,axs  = plt.subplots(n_row, n_col, figsize=(13.1*cm,3.5*n_row*cm),
                            gridspec_kw=gridspec)

    for i,terminal in enumerate(terminalsfig6_1):
        for j,model in zip([0,1]*4 ,models_):
            globals().update(model)
            globals().update(np.load(data_file(figname), allow_pickle=True))
            globals().update(settings.item())
            df = dict(np.load(data_file(figname), allow_pickle=True))
            x1 = -5
            I = (data[0]>=x1+2) & (data[0]<=num_mins*60)
            T,Rp,ADP,ATP,ARG = data[[0,2,3,4,5]][:,I]
            pi = Pt - ADP - 2*ATP - (Pht - ARG )
            ΔG0 = -30.6e3 
            Tmp = 300 
            ΔG  = ΔG0 + R*Tmp*np.log((ADP*10**-3)*(pi*10**-3)/(ATP*10**-3))
            ΔG *= 1e-3 


            # Show production caps as arrow heads at appropriate height along the y axis.
            # If they're too large, have the arrow head point up and write the value.
            style = ArrowStyle('wedge', tail_width=1.5, shrink_factor=0.5)
            opt   = dict(mutation_scale=1, arrowstyle=style, color=colors[terminal])
            dx    = 50
            ax    = axs[3,j]
            if Kph == None:
                k = 1
            else:
                k = 0     
            ## Graphing other curves using 0.154 cap
            if rate_cap_per_pct_mito == caps[2]:            
                axs[1,j].plot(T, ATP, **term_opt(terminal))
                axs[2,j].plot(T,ATP/ADP,**term_opt(terminal))
                axs[3,j].plot(T, Rp, **term_opt(terminal))
                acc = np.gradient(Rp, T)
                axs[4,j].plot(T, acc, **term_opt(terminal))
                axs[5,j].plot(T, -ΔG, **term_opt(terminal))  
                if terminal == 'M6Ib':
                    ax.add_patch(FancyArrowPatch((x1+dx+40,rate_cap), (x1+40,rate_cap), **opt))
                if terminal == 'M13Ib':
                    ax.add_patch(FancyArrowPatch((x1+dx+15,rate_cap), (x1+15,rate_cap), **opt))

            elif terminal == 'M6Ib' and rate_cap_per_pct_mito == caps[1]:
                axs[1,j].plot(T, ATP, color='grey')            
            elif terminal == 'M13Ib' and rate_cap_per_pct_mito == caps[1]:
                axs[1,j].plot(T, ATP, color='black')            

            axs[1,j].set_ylim(*ATP[0]*np.array([-0.,1.2]))
            axs[2,j].set_ylim(*(ATP/ADP)[0]*np.array([-0.,1.2]))
        I = times<=num_mins*60
        axs[0,0].set_ylim((0,1.5))
        axs[0,0].plot(times[I], consumption[I], **term_opt(terminal))
        axs[0,1].plot(times[I], consumption[I], **term_opt(terminal))
    axs_ = axs[0,:1].tolist() + [ax for col in axs.T for ax in col[1:]]
    for i,ax in enumerate(axs_):
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)
        ax.set_xlabel('Time (s)')
        ax.locator_params(nbins=4)
        ax.text(s=chr(65+i), transform=ax.transAxes, **panel_opt)
        ax.set_xlim((-2,num_mins*60),None)
    axs[0,1].set_xlim((-0.1,2.9),None)
    axs[0,0].set_ylabel('ATP cons. rate (mM/s)')
    axs[0,0].legend(frameon=False,ncol = 2)
    axs[0,1].spines.right.set_visible(False)
    axs[0,1].spines.top.set_visible(False)
    axs[0,1].set_xlabel('Time (s)')
    for j in range(axs.shape[1]):
        axs[1,j].set_ylabel('[ATP] (mM)')
        axs[2,j].set_ylabel('[ATP]/[ADP] ')
        axs[3,j].set_ylabel('ATP prod. rate \n(mM/s)')
        axs[4,j].set_ylabel('ATP prod. accel. \n(mM/s/s)')
        axs[5,j].set_ylabel('ATP hydrolysis\nenergy (kJ/mol)')
    for i in [3,4,5]:
        ylim = list(zip(*[ax.get_ylim() for ax in axs[i]]))
        ylim = min((0,)+ylim[0]), max(ylim[1])
        for ax in axs[i]:
            ax.set_ylim(ylim)
    axs[0,0].set_yticks([0,0.5,1.0,1.5])
    axs[0,1].set_yticks([0,0.2,0.4,0.6,0.8])    
    axs[5,0].set_yticks([15,30,45,60])    
    axs[5,1].set_yticks([15,30,45,60])    

    fig.suptitle(figname + ' w/ (left column) + w/o Arg (right)' )
    savefig(figname)
    plt.show()

## All Individualized Consumption Curves

In [ ]:
fn_figS1_fnlist = [fn_fig7_6bs,fn_fig7_6bs ,fn_fig6_1 ,fn_fig6_1]  ## collecting fignames to draw from
fn_figS1_modellist = [*models_fig7_6bs ]
fn_figS1_modellist[-1] = models_fig6_1[-1]
num_minsS1   =  [1/60, 20 ]
terminals_figS1 = [*terminalsfig7[:-1], *terminalsfig6_1]

fn_figS1 = 'figS1_consumptions'


In [ ]:
fn_figS1_fnlist,len(fn_figS1_modellist), terminalsfig7[:-1], terminals_figS1,models_fig6_1

In [ ]:
#fn_figS1_fnlist,fn_figS1_modellist

In [ ]:
# for figname,models_ in zip( fn_figS1_fnlist[:2],fn_figS1_modellist[:2]):

#     n_row    = 1
#     n_col    = int(len(terminals_figS1))
#     gridspec = dict(hspace=0.55, wspace=0.55, left=0.08,
#                     right=0.98, bottom=0.07, top=0.95)
#     fig,axs  = plt.subplots(n_row, n_col, figsize=(13.1*cm,3.5*n_row*cm),
#                             gridspec_kw=gridspec)
#     for i,terminal in enumerate([*terminals_figS1][:2]):
#         globals().update(np.load(data_file(figname), allow_pickle=True))
#         df = dict(np.load(data_file(figname), allow_pickle=True))
#         x1 = -5
#         I = (data[0]>=x1+2) & (data[0]<=num_mins*60)
#         T,Rp,ADP,ATP,ARG = data[[0,2,3,4,5]][:,I]
#         I = times<=num_mins*60
#         axs[i].set_ylim((0,1.5))
#         axs[i].plot(times[I], consumption[I], **term_opt(terminal))
#         axs[i].plot(times[I], consumption[I], **term_opt(terminal))
# for figname,models_ in zip( fn_figS1_fnlist[2:],fn_figS1_modellist[2:]):

#     n_row    = 1
#     n_col    = int(len(terminals_figS1))
#     gridspec = dict(hspace=0.55, wspace=0.55, left=0.08,
#                     right=0.98, bottom=0.07, top=0.95)
#     fig,axs  = plt.subplots(n_row, n_col, figsize=(13.1*cm,3.5*n_row*cm),
#                             gridspec_kw=gridspec)
#     for i,terminal in enumerate([*terminals_figS1][2:]):
#         globals().update(np.load(data_file(figname), allow_pickle=True))
#         df = dict(np.load(data_file(figname), allow_pickle=True))
#         x1 = -5
#         I = (data[0]>=x1+2) & (data[0]<=num_mins*60)
#         T,Rp,ADP,ATP,ARG = data[[0,2,3,4,5]][:,I]
#         I = times<=num_mins*60
#         axs[i].set_ylim((0,1.5))
#         axs[i].plot(times[I], consumption[I], **term_opt(terminal))
#         axs[i].plot(times[I], consumption[I], **term_opt(terminal))
# plt.show()


In [ ]:
n_row    = 1
n_col    = int(len(terminals_figS1))
gridspec = dict(hspace=0.05, wspace=0.05, left=0.08,
                right=0.98, bottom=0.07, top=0.85)
fig,axs  = plt.subplots(n_row, n_col, layout="constrained", figsize=(2*13.1*cm,1.5*3.5*n_row*cm),
                        gridspec_kw=gridspec)
for i,terminal,models_,figname in zip(np.arange(0,n_col,1) ,[*terminals_figS1],fn_figS1_modellist,fn_figS1_fnlist):
    globals().update(np.load(data_file(figname), allow_pickle=True))
    df = dict(np.load(data_file(figname), allow_pickle=True))
    x1 = -5
    I = (data[0]>=x1+2) & (data[0]<=num_mins*60)
    T = data[[0]][:,I]
    I = times<=num_mins*60
    axs[i].plot(times[I], consumption[I], **term_opt(terminal))
    axs[i].plot(times[I], consumption[I], **term_opt(terminal))
yticks = np.linspace(0,2.5,6)
for ax in axs:
    ax.set_yticks(yticks)  
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.set_xlabel('Time (s)')
axs[0].set_ylabel('ATP cons. rate (mM/s)')
plt.suptitle("Consumption Curves for Figs. 6A, 7A")
savefig(fn_figS1)
plt.show()
